In [2]:
import math
import re
from dataclasses import dataclass, asdict
from itertools import combinations
from typing import Tuple, Type, TypeVar
from collections import defaultdict
from functools import lru_cache

import numpy as np
import pandas as pd

In [2]:
searched_sum = 2020

In [3]:
%%timeit
with open("day_01_input.txt") as input_data:
    for i, pair in enumerate(combinations(sorted((int(x) for x in input_data)), 2)):
        if sum(pair) == searched_sum:
            day_01_p1 = math.prod(pair)
            break

212 µs ± 9.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
%%timeit
with open("day_01_input.txt") as input_data:
    for i, triple in enumerate(combinations(sorted((int(x) for x in input_data)), 3)):
        if sum(triple) == searched_sum:
            day_01_p2 = math.prod(triple)
            break

8.99 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
parse_expression = re.compile(r"^(\d+)-(\d+)\s+(\w):\s+(.*)$")

def parse_entry(entry: str) -> Tuple:
    g = parse_expression.match(entry).groups()
    return *map(int, g[:2]), *g[2:]

In [6]:
%%timeit
with open("day_02_input.txt") as input_data:
    day_02_p1 = sum(f <= p.count(c) <= s for f, s, c, p in map(parse_entry, input_data))

1.24 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%%timeit
with open("day_02_input.txt") as input_data:
    day_02_p2 = sum(
        (p[f - 1] == c) != (p[s - 1] == c)
        for f, s, c, p in map(parse_entry, input_data)
    )

1.16 ms ± 33.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]

In [9]:
%%timeit
with open("day_03_input.txt") as input_data:
    tree_map = np.array(
        [np.array([c == "#" for c in row.strip()], dtype=bool) for row in input_data]
    )
height, width = tree_map.shape

day_03_p1 = np.sum(tree_map[(np.arange(height), np.arange(0, height * 3, 3) % width)])

1.34 ms ± 43.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit
with open("day_03_input.txt") as input_data:
    tree_map = np.array(
        [np.array([c == "#" for c in row.strip()], dtype=bool) for row in input_data]
    )
height, width = tree_map.shape

slope_index_generator = (
    (np.arange(0, height, d), np.arange(0, math.ceil(height / d) * r, r) % width)
    for r, d in slopes
)
day_03_p2 = math.prod(np.sum(tree_map[s]) for s in slope_index_generator)

1.44 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
# noinspection PyTypeChecker
T = TypeVar("T", bound="Passport")

@dataclass
class Passport:
    byr: str = ""
    iyr: str = ""
    eyr: str = ""
    hgt: str = ""
    hcl: str = ""
    ecl: str = ""
    pid: str = ""
    cid: str = ""

    @classmethod
    def from_string(cls: Type[T], info: str) -> T:
        # I guess this an IDE bug as creating a dict from a generator should be valid
        # noinspection PyTypeChecker
        return Passport(
            **dict(p.split(":") for p in info.strip().replace("\n", " ").split())
        )

    def is_valid(self, excludes: list[str] = ("cid",), strict: bool = False) -> bool:
        def check_year_bounds(year: str, min_bound: int, max_bound: int) -> bool:
            return year.isdigit() and min_bound <= int(year) <= max_bound

        # noinspection PyShadowingNames
        def check_height(height: str) -> bool:
            match = re.match(r"^(\d{2,3})(cm|in)$", height)
            if match:
                value, unit = match.groups()
                value = int(value)
                return 150 <= value <= 193 if unit == "cm" else 59 <= value <= 76
            return False

        complete = all(v != "" for k, v in asdict(self).items() if k not in excludes)
        if strict:
            return (
                complete
                and check_year_bounds(self.byr, 1920, 2002)
                and check_year_bounds(self.iyr, 2010, 2020)
                and check_year_bounds(self.eyr, 2020, 2030)
                and check_height(self.hgt)
                and bool(re.match(r"^#[0-9a-f]{6}$", self.hcl))
                and self.ecl in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
                and bool(re.match(r"^\d{9}$", self.pid))
            )

        return complete

In [12]:
%%timeit
with open("day_04_input.txt") as input_data:
    passport_infos = input_data.read().strip().split("\n\n")

day_04_p1 = sum(Passport.from_string(s).is_valid() for s in passport_infos)

4.96 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
with open("day_04_input.txt") as input_data:
    passport_infos = input_data.read().strip().split("\n\n")

day_04_p2 = sum(Passport.from_string(s).is_valid(strict=True) for s in passport_infos)

5.92 ms ± 73.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
def seat_id(input_string: str) -> int:
    return int(input_string.translate(str.maketrans("FLBR", "0011")), 2)

In [15]:
%%timeit
with open("day_05_input.txt") as input_data:
    seat_ids = sorted(map(seat_id, input_data))
day_05_p1 = seat_ids[-1]

636 µs ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%%timeit
with open("day_05_input.txt") as input_data:
    seat_ids = sorted(map(seat_id, input_data))
day_05_p2 = seat_ids[np.nonzero(np.diff(seat_ids) == 2)[0][0]] + 1

702 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
%%timeit
with open("day_06_input.txt") as input_data:
    groups = [{q for q in g.replace("\n", "")} for g in input_data.read().split("\n\n")]
day_06_p1 = sum(len(x) for x in groups)

814 µs ± 9.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
%%timeit
with open("day_06_input.txt") as input_data:
    groups = [[set(q) for q in g.split("\n")] for g in input_data.read().split("\n\n")]
day_06_p2 = sum(len(set.intersection(*[y for y in x])) for x in groups)

1.67 ms ± 78.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
bag_regex = r"^(\w+ \w+)"
contained_regex = r"(\d+) (\w+ \w+)"


def parse_rule(input_string: str) -> tuple:
    bag_match = re.match(bag_regex, input_string).group(1)
    contained_match = [m.groups() for m in re.finditer(contained_regex, input_string)]
    return bag_match, [(int(m[0]), m[1]) for m in contained_match]

In [20]:
%%timeit
with open("day_07_input.txt") as input_data:
    bag_map = dict(map(parse_rule, input_data))

@lru_cache()
def contains_bag(query_bag: str, bag_to_check: str) -> bool:
    contained_bags = {b for _, b in bag_map[bag_to_check]}
    if not contained_bags:
        return False
    if query_bag in contained_bags:
        return True
    return any(contains_bag(query_bag, b) for b in contained_bags)


day_07_p1 = sum(contains_bag("shiny gold", s) for s in bag_map)

7.6 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit
with open("day_07_input.txt") as input_data:
    bag_map = dict(map(parse_rule, input_data))

lookup_dict = defaultdict(set)
for bag, contained_bags in bag_map.items():
    for _, contained_bag in contained_bags:
        lookup_dict[contained_bag].add(bag)

bags_containing_shiny_gold = lookup_dict["shiny gold"]
bags_to_process = bags_containing_shiny_gold.copy()
while bags_to_process:
    new_bags = lookup_dict[bags_to_process.pop()]
    bags_containing_shiny_gold |= new_bags
    bags_to_process |= new_bags

day_07_p1 = len(bags_containing_shiny_gold)

3.02 ms ± 9.83 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
with open("day_07_input.txt") as input_data:
    bag_map = dict(map(parse_rule, input_data))

def get_sub_bag_count(count: int, bag: str) -> int:
    if not bag_map[bag]:
        return count
    return count + count * sum(get_sub_bag_count(c, b) for c, b in bag_map[bag])


day_07_p1 = get_sub_bag_count(1, 'shiny gold') - 1

2.49 ms ± 37.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
position = 0
visited = set()
accumulator = 0

class LoopException(Exception):
    pass


def noop(*args) -> None:
    global position
    if position in visited:
        raise LoopException
    visited.add(position)
    position += 1


def acc(value: int) -> None:
    global position
    global accumulator
    if position in visited:
        raise LoopException
    visited.add(position)
    accumulator += value
    position += 1


def jmp(value: int) -> None:
    global position
    if position in visited:
        raise LoopException
    visited.add(position)
    position += value


op_map = {
    "nop": noop,
    "acc": acc,
    "jmp": jmp
}

In [24]:
%%timeit
with open("day_08_input.txt") as input_data:
    op_codes = [x.strip().split() for x in input_data]

callstack = []
while True:
    callstack.append(position)
    op, arg = op_codes[position]
    try:
        op_map[op](int(arg))
    except LoopException:
        day_08_p1 = accumulator
        break

140 µs ± 5.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%%timeit
with open("day_08_input.txt") as input_data:
    op_codes = [x.strip().split() for x in input_data]

position = 0
visited = set()
accumulator = 0
callstack = []
while True:
    callstack.append(position)
    op, arg = op_codes[position]
    try:
        op_map[op](int(arg))
    except LoopException:
        break

position_to_check = [x for x in callstack[:-1] if op_codes[x][0] in ["nop", "jmp"]]

terminated = False
for corruption_candidate in position_to_check:
    if terminated:
        break
    position = 0
    visited = set()
    accumulator = 0
    altered_op_codes = op_codes.copy()
    old_op, arg = altered_op_codes[corruption_candidate]
    altered_op_codes[corruption_candidate] = ("nop", arg) if old_op == "jmp" else ("jmp", arg)
    while True:
        if position >= len(altered_op_codes):
            day_08_p2 = accumulator
            terminated = True
            break
        op, arg = altered_op_codes[position]
        try:
            op_map[op](int(arg))
        except LoopException:
            break

154 µs ± 8.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%%timeit
offset = 25
with open("day_09_input.txt") as input_data:
    numbers = np.array([int(x) for x in input_data])

x = np.lib.stride_tricks.as_strided(numbers, (len(numbers) - offset, offset), (numbers.strides[0], numbers.strides[0]))

for c, p in zip(numbers[25:], np.expand_dims(x, axis=1)):
    if not (p + p.T == c).any():
        day_09_p1 = c
        break

3.53 ms ± 569 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
with open("day_09_input.txt") as input_data:
    numbers = np.array([int(x) for x in input_data])

offset = 25
x = np.lib.stride_tricks.as_strided(numbers, (len(numbers) - offset, offset), (numbers.strides[0], numbers.strides[0]))

for c, p in zip(numbers[25:], x):
    if not (p.reshape(-1, 1) + p.reshape(1, -1) == c).any():
        day_09_p1 = c
        break

accumulated = numbers.cumsum().reshape(1, -1)
for left, right in zip(*np.nonzero((accumulated - accumulated.T) == day_09_p1)):
    if right - left > 2:
        involved = numbers[left + 1:right + 1]
        day_09_p2 = min(involved) + max(involved)

6.22 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
%%timeit
with open("day_10_input.txt") as input_data:
    adapters = np.sort(np.array([int(x) for x in input_data]))

differences = np.diff(np.hstack(([0], adapters, [adapters[-1] + 3])))
day_10_p1 = np.sum(differences == 1) * (np.sum(differences == 3))

84.6 µs ± 6.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
%%timeit
with open("day_10_input.txt") as input_data:
    adapters = np.sort(np.array([int(x) for x in input_data]))

differences = np.diff(np.hstack(([0], adapters, [adapters[-1] + 3])))
run_diffs = np.diff(np.hstack(([0], differences == 1, [0])))
run_starts, = np.nonzero(run_diffs > 0)
run_ends, = np.nonzero(run_diffs < 0)

lookup = {2: 2, 3: 4, 4: 7}

day_10_p2 = math.prod(lookup[width] for start, end in zip(run_starts, run_ends) if (width := end - start) > 1)


117 µs ± 8.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
lookup = {"L": 0, "#": 1, ".": np.nan}

1.19 ms ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%%timeit
with open("day_11_input.txt") as input_data:
    seat_map = np.array([[lookup[c] for c in line.strip()] for line in input_data])

floor = np.isnan(seat_map)
height, width = seat_map.shape

seat_map = np.zeros((height + 2, width + 2), dtype=bool)
seat_view = seat_map[1:-1, 1:-1]

neighbour_map = np.lib.stride_tricks.as_strided(
    seat_map,
    shape=(height, width, 3, 3),
    strides=seat_map.strides + seat_map.strides,
)

day_11_p1 = -1
while day_11_p1 != (current_value := np.sum(seat_view, dtype=int)):
    neighbour_sums = np.sum(neighbour_map, axis=(2, 3), dtype=np.uint8)
    seat_view[(neighbour_sums == 0) & ~floor] = True
    seat_view[(neighbour_sums > 4) & seat_view & ~floor] = False
    day_11_p1 = current_value

39.6 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%%timeit
with open("day_12_input.txt") as input_data:
    directions = [(line[0], int(line[1:].strip())) for line in input_data]

position = np.array([0, 0])
direction = np.array([0, 1])

direction_lookup = {
    "N": np.array([-1, 0]),
    "E": np.array([0, 1]),
    "S": np.array([1, 0]),
    "W": np.array([0, -1]),
}
# [[cos a, -sin a], [sin a, cos a]]
rotation_lookup = {
    90: np.array([[0, -1], [1, 0]]),
    180: np.array([[-1, 0], [0, -1]]),
    270: np.array([[0, 1], [-1, 0]]),
}

for command, value in directions:
    if command in "NESW":
        position += value * direction_lookup[command]
    elif command in "RL":
        rotation_value = value if command == "R" else 360 - value
        direction = np.matmul(direction, rotation_lookup[rotation_value])
    elif command == "F":
        position += value * direction
    else:
        print("Unsupported command", command)


day_12_p1 = np.sum(np.abs(position))

1.45 ms ± 9.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
%%timeit
with open("day_12_input.txt") as input_data:
    directions = [(line[0], int(line[1:].strip())) for line in input_data]

direction_lookup = {
    "N": np.array([-1, 0]),
    "E": np.array([0, 1]),
    "S": np.array([1, 0]),
    "W": np.array([0, -1]),
}
# [[cos a, -sin a], [sin a, cos a]]
rotation_lookup = {
    90: np.array([[0, -1], [1, 0]]),
    180: np.array([[-1, 0], [0, -1]]),
    270: np.array([[0, 1], [-1, 0]]),
}

position = np.array([0, 0])
waypoint = np.array([-1, 10])

for command, value in directions:
    if command in "NESW":
        waypoint += value * direction_lookup[command]
    elif command in "RL":
        rotation_value = value if command == "R" else 360 - value
        waypoint = np.matmul(waypoint, rotation_lookup[rotation_value])
    elif command == "F":
        position += value * waypoint
    else:
        print("Unsupported command", command)

day_12_p2 = np.sum(np.abs(position))

1.36 ms ± 28.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%%timeit
with open("day_13_input.txt") as input_data:
    earliest_arrival = int(input_data.readline().strip())
    bus_ids = [int(x) for x in input_data.readline().split(",") if x != "x"]

wait_time, bus_id = sorted((bus_id - earliest_arrival % bus_id, bus_id) for bus_id in bus_ids)[0]

day_13_p1 = wait_time * bus_id

23.8 µs ± 3.88 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%%timeit
with open("day_13_input.txt") as input_data:
    input_data.readline()
    bus_ids = [(offset, int(x)) for offset, x in enumerate(input_data.readline().split(",")) if x != "x"]

step_size = bus_ids.pop(0)[1]
position = step_size
while bus_ids:
    offset, bus_id = bus_ids.pop(0)
    while (position + offset) % bus_id != 0:
        position += step_size
    step_size *= bus_id

day_13_p2 =position

62.6 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%%timeit
mask_regex = re.compile(r"^mask\s+=\s+([X10]+)")
mem_regex = re.compile(r"^mem\[(\d+)\]\s=\s(\d+)")

result = {}
with open("day_14_input.txt") as input_data:
    for line in input_data:
        if mask_match := mask_regex.match(line):
            and_pattern = int(mask_match.group(1).replace("1", "0").replace("X", "1"), 2)
            or_pattern = int(mask_match.group(1).replace("X", "0"), 2)
        else:
            position, value = map(int, mem_regex.match(line).groups())
            result[position] = value & and_pattern | or_pattern

day_14_p1 = sum(result.values())

780 µs ± 50.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit
mask_regex = re.compile(r"^mask\s+=\s+([X10]+)")
mem_regex = re.compile(r"^mem\[(\d+)\]\s=\s(\d+)")
result = {}
with open("day_14_input.txt") as input_data:
    for line in input_data:
        if mask_match := mask_regex.match(line):
            and_pattern = int(mask_match.group(1).replace("0", "1").replace("X", "0"), 2)
            or_template = mask_match.group(1)
            or_patterns = []
            x_count = line.count("X")
            for x in range(2**x_count):
                or_pattern = or_template
                for c in bin(x)[2:].zfill(x_count):
                    or_pattern = or_pattern.replace("X", c, 1)
                or_patterns.append(int(or_pattern, 2))
        else:
            position, value = map(int, mem_regex.match(line).groups())
            for or_pattern in or_patterns:
                result[position & and_pattern | or_pattern] = value

day_14_p2 = sum(result.values())


36 ms ± 212 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
input_data = [6, 3, 15, 13, 1, 0]

history = {x: i + 1 for i, x in enumerate(input_data[:-1])}
last_value = input_data[-1]
turn = len(input_data) + 1
last_turn = 2020
while turn <= last_turn:
    if last_value not in history:
        current_value = 0
    else:
        current_value = turn - history[last_value] - 1
    history[last_value] = turn - 1
    last_value = current_value
    turn += 1

day_14_p1 = last_value

373 µs ± 72.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%%timeit
input_data = [6, 3, 15, 13, 1, 0]

history = {x: i + 1 for i, x in enumerate(input_data[:-1])}
last_value = input_data[-1]
turn = len(input_data) + 1
last_turn = 30000000
while turn <= last_turn:
    if last_value not in history:
        current_value = 0
    else:
        current_value = turn - history[last_value] - 1
    history[last_value] = turn - 1
    last_value = current_value
    turn += 1

day_15_p2 = last_value

9.36 s ± 437 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
with open("day_16_input.txt") as input_data:
    rules_dict = {}
    while match := re.match(r"^((?:\w+\s?)+):\s(\d+-\d+)\sor\s(\d+-\d+)", input_data.readline()):
        rules_dict[match.group(1)] = [pd.Interval(*map(int, g.split("-")), closed="both") for g in match.groups()[1:]]
    while "your ticket:" not in input_data.readline():
        pass
    my_ticket = list(map(int, input_data.readline().split(",")))
    while "nearby tickets:" not in input_data.readline():
        pass
    nearby_tickets = [list(map(int, x.split(","))) for x in input_data]


def check_valid(ticket: list[int]) -> tuple[bool, int]:
    valid = True
    error_rate = 0
    for value in ticket:
        if not any(value in rule for name, rules in rules_dict.items() for rule in rules):
            valid = False
            error_rate += value
    return valid, error_rate


day_16_p1 = sum(check_valid(ticket)[1] for ticket in nearby_tickets)

7.18 ms ± 998 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
with open("day_16_input.txt") as input_data:
    rules_dict = {}
    while match := re.match(r"^((?:\w+\s?)+):\s(\d+-\d+)\sor\s(\d+-\d+)", input_data.readline()):
        rules_dict[match.group(1)] = [pd.Interval(*map(int, g.split("-")), closed="both") for g in match.groups()[1:]]
    while "your ticket:" not in input_data.readline():
        pass
    my_ticket = list(map(int, input_data.readline().split(",")))
    while "nearby tickets:" not in input_data.readline():
        pass
    nearby_tickets = [list(map(int, x.split(","))) for x in input_data]


def check_valid(ticket: list[int]) -> tuple[bool, int]:
    valid = True
    error_rate = 0
    for value in ticket:
        if not any(value in rule for name, rules in rules_dict.items() for rule in rules):
            valid = False
            error_rate += value
    return valid, error_rate

valid_tickets = [ticket for ticket in nearby_tickets if check_valid(ticket)[0]]


def map_fields(ticket: list[int]) -> list[set]:
    possible_fields = [set() for _ in range(len(ticket))]
    for i, value in enumerate(ticket):
        possible_fields[i] = {name for name, rules in rules_dict.items() for rule in rules if value in rule}

    return possible_fields


candidates = {i: f[0].intersection(*f) for i, f in enumerate(zip(*[map_fields(ticket) for ticket in valid_tickets]))}
candidates_sorted = {k: v for k, v in sorted(candidates.items(), key=lambda item: len(item[1]))}

assigned = {}
for i, fields in candidates_sorted.items():
    fields = fields - assigned.keys()
    if len(fields) == 1:
        field, = fields
        assigned[field] = i
    else:
        print("Should not happen")

day_16_p2 = math.prod(my_ticket[v] for k, v in assigned.items() if k.startswith('departure'))

66.5 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit
with open("day_17_input.txt") as input_data:
    initial_plane = np.array(
        [[c == "#" for c in line.strip()] for line in input_data], dtype=bool
    )[np.newaxis, :, :]

space = np.pad(initial_plane, 7, constant_values=False)

depth, height, width = [x - 2 for x in space.shape]

space_view = space[1 : depth + 1, 1 : height + 1, 1 : width + 1]

neighbour_map = np.lib.stride_tricks.as_strided(
    space,
    shape=(depth, height, width, 3, 3, 3),
    strides=space.strides + space.strides,
)

for i in range(6):
    swap_space = space_view.copy()
    neighbour_sums = np.sum(neighbour_map, axis=(3, 4, 5), dtype=np.uint8)

    swap_space[~((neighbour_sums == 3) | (neighbour_sums == 4)) & space_view] = False
    swap_space[(neighbour_sums == 3) & ~space_view] = True

    space_view[:, :, :] = swap_space

day_17_p1 = np.sum(space_view)

7.66 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit
with open("day_17_input.txt") as input_data:
    initial_plane = np.array(
        [[c == "#" for c in line.strip()] for line in input_data], dtype=bool
    )[np.newaxis, np.newaxis, :, :]

space = np.pad(initial_plane, 7, constant_values=False)

time, depth, height, width = [x - 2 for x in space.shape]

space_view = space[1 : time + 1, 1 : depth + 1, 1 : height + 1, 1 : width + 1]

neighbour_map = np.lib.stride_tricks.as_strided(
    space,
    shape=(time, depth, height, width, 3, 3, 3, 3),
    strides=space.strides + space.strides,
)

for i in range(6):
    swap_space = space_view.copy()
    neighbour_sums = np.sum(neighbour_map, axis=(4, 5, 6, 7), dtype=np.uint8)

    swap_space[~((neighbour_sums == 3) | (neighbour_sums == 4)) & space_view] = False
    swap_space[(neighbour_sums == 3) & ~space_view] = True

    space_view[:, :, :, :] = swap_space

day_17_p2 = np.sum(space_view)

249 ms ± 3.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
